In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Installing pyspark
!pip install pyspark
!pip install findspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Spark Application').getOrCreate()

In [ ]:
# df = spark.read.csv("/kaggle/input/us-accidents/US_Accidents_Dec20_Updated.csv",inferSchema=True, header=True)
df = spark.read.csv("/kaggle/input/us-accidents/US_Accidents_Dec20_Updated.csv",inferSchema=True, header=True).limit(1500000)

In [ ]:
df.head()

In [ ]:
df.printSchema()

### Preprocessing and Data Explorations

In [ ]:
from pyspark.sql.functions import *

In [ ]:
cols_to_drop = ['ID','Start_Time','End_Time','Description','Street',
                'Weather_Timestamp','Zipcode','County','City','Airport_Code','Precipitation(in)'] 
# Weather_Condition to be processed again after cleaning

In [ ]:
df = df.drop(*cols_to_drop)

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [ ]:
# Dict_Null = {col:df.filter(isnan(df[col[0]])).count() for col in df.dtypes if col[1] != 'boolean'}
# Dict_Null

In [ ]:
# Replace nan with Nulls
columns = df.dtypes
for cols, typ in columns:
    if typ != 'boolean':
        df = df.withColumn(cols,when(isnan(col(cols)),None).otherwise(col(cols)))

In [ ]:
# Dict_Null = {col:df.filter(isnan(df[col[0]])).count() for col in df.dtypes if col[1] != 'boolean'}
# Dict_Null

In [ ]:
# Dict_Null2 = {col:df.filter(isnull(df[col[0]])).count() for col in df.dtypes }
# Dict_Null2

In [ ]:
label = 'Severity'
string_cols =  [cols[0] for cols in df.dtypes if cols[1] == "string" ]
num_cols = [cols[0] for cols in df.dtypes if cols[1] == "int" or cols[1] == "double" ]
num_cols.remove(label)
bool_cols = [cols[0] for cols in df.dtypes if cols[1] == "boolean"]

# string_cols,
# num_cols
# bool_cols

In [ ]:
df = df.fillna("unknown",string_cols)
df = df.fillna(0,num_cols)

In [ ]:
%%time 
Dict_Null3 = {col:df.filter(isnull(df[col[0]])).count() for col in df.dtypes }
Dict_Null3

In [ ]:
for c in bool_cols:
    df = df.withColumn(c,col(c).cast("integer"))    

In [ ]:
(train, test) = df.randomSplit([0.7, 0.3])
train.count(), test.count()
# df.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
sIndexer = [StringIndexer(inputCol=cols, outputCol=cols+"Index") for cols in  string_cols]

In [ ]:
assembler = VectorAssembler(inputCols=[s.getOutputCol() for s in sIndexer]+bool_cols+num_cols, outputCol='features')

In [ ]:
# reg = 0,
lr = LogisticRegression(featuresCol="features", labelCol=label)

In [ ]:
pipeline = Pipeline(stages=sIndexer+[assembler, lr])
model = pipeline.fit(train)


In [ ]:
preds = model.transform(train)
print("Prediction")
preds.select("Severity","prediction").show(20)

# evaluate the accuracy of the model using the test set
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', labelCol="Severity")
accuracy = evaluator.evaluate(preds)

print()
print('#####################################')
print(f"Accuracy is {accuracy}")
print('#####################################')
print()


In [ ]:
!pip install onnx
!pip install onnxmltools
!pip install mlflow
!pip install onnxruntime

In [ ]:
from onnxmltools import convert_sparkml
from onnxmltools.convert.sparkml.utils import buildInitialTypesSimple
from onnxmltools.convert.common.data_types import StringTensorType, FloatTensorType
import mlflow.onnx

In [ ]:
import os
if not os.path.exists('model_onnx/'):
    os.mkdir('model_onnx/')
    
if not os.path.exists('model_spark/'):
    os.mkdir('model_spark/')
    
if not os.path.exists('model_mlflow'):
    os.mkdir('model_mlflow')

In [ ]:
%%time
# Native ONNX framework
initial_types = buildInitialTypesSimple(df.drop('Severity'))
# initial_types
onnx_model = convert_sparkml(model, 'Pyspark model', initial_types, spark_session = spark)

In [ ]:
%%time
with open(os.path.join("model_onnx/", "model.onnx"), "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
%%time 
# Pyspark native
model.save('model_spark/spark_model')

In [ ]:
%%time

# ONNX on MLFlow
mlflow.onnx.save_model(onnx_model,"model_mlflow/mlflow_model")

In [ ]:
!ls -ltrh

In [ ]:
!pip install py-cpuinfo

In [ ]:
import platform
platform.processor()
# import cpuinfo

# 'Intel64 Family 6 Model 23 Stepping 6, GenuineIntel'

In [ ]:
import cpuinfo
cpuinfo.get_cpu_info()['brand_raw']

In [ ]:
# from pyspark.sql.functions import array_contains, col, explode
# def BinaryEncoder(df, column_name):
#     col_vals = [
#         x[0] for x in 
#         df.select(column_name).distinct().orderBy(column_name).collect()]
#     print(col_vals)
    
    
    
# %time BinaryEncoder(df_1, "Side")